# CSE 252B: Computer Vision II, Winter 2019 – Assignment 3
### Instructor: Ben Ochoa
### Due: Wednesday, February 20, 2019, 11:59 PM

## Instructions
* Review the academic integrity and collaboration policies on the course website.
* This assignment must be completed individually.
* This assignment contains both math and programming problems.
* All solutions must be written in this notebook
* Math problems must be done in Markdown/LATEX. Remember to show work and describe your solution.
* Programming aspects of this assignment must be completed using Python in this notebook.
* Your code should be well written with sufficient comments to understand, but there is no need to write extra markdown to describe your solution if it is not explictly asked for.
* This notebook contains skeleton code, which should not be modified (This is important for standardization to facilate effeciant grading).
* You may use python packages for basic linear algebra, but you may not use packages that directly solve the problem. Ask the instructor if in doubt.
* You must submit this notebook exported as a pdf. You must also submit this notebook as an .ipynb file.
* Your code and results should remain inline in the pdf (Do not move your code to an appendix).
* You must submit both files (.pdf and .ipynb) on Gradescope. You must mark each problem on Gradescope in the pdf.
* It is highly recommended that you begin working on this assignment early.

## Problem 1 (Programming):  Estimation of the Camera Pose - Outlier rejection (20 points)
  Download input data from the course website.  The file
  hw3_points3D.txt contains the coordinates of 60 scene points
  in 3D (each line of the file gives the $\tilde{X}_i$, $\tilde{Y}_i$,
  and $\tilde{Z}_i$ inhomogeneous coordinates of a point).  The file
  hw3_points2D.txt contains the coordinates of the 60
  corresponding image points in 2D (each line of the file gives the
  $\tilde{x}_i$ and $\tilde{y}_i$ inhomogeneous coordinates of a
  point).  The corresponding 3D scene and 2D image points contain both
  inlier and outlier correspondences.  For the inlier correspondences,
  the scene points have been randomly generated and projected to image
  points under a camera projection matrix (i.e., $\boldsymbol{x}_i = \boldsymbol{P}
  \boldsymbol{X}_i$), then noise has been added to the image point
  coordinates.

  The camera calibration matrix was calculated for a $1280 \times 720$
  sensor and $45\,^\circ$ horizontal field of view lens.  The
  resulting camera calibration matrix is given by
  
  $\boldsymbol{K} = \left[
    \begin{array}{c c c}
      1545.0966799187809 & 0 & 639.5\\
      0 & 1545.0966799187809 & 359.5\\
      0 & 0 & 1
    \end{array}\right]$
    
  For each image point $\boldsymbol{x} = (x, y, w)^\top = (\tilde{x},
  \tilde{y}, 1)^\top$, calculate the point in normalized coordinates
  $\hat{\boldsymbol{x}} = \boldsymbol{K}^{-1} \boldsymbol{x}$.

  Determine the set of inlier point correspondences using the
  M-estimator Sample Consensus (MSAC) algorithm, where the maximum
  number of attempts to find a consensus set is determined adaptively.
  For each trial, use the 3-point algorithm of Finsterwalder (as
  described in the paper by Haralick et al.) to estimate the camera
  pose (i.e., the rotation $\boldsymbol{R}$ and translation $\boldsymbol{t}$ from the
  world coordinate frame to the camera coordinate frame), resulting in
  up to 4 solutions, and calculate the error and cost for each
  solution.  Note that the 3-point algorithm requires the 2D points in
  normalized coordinates, not in image coordinates.  Calculate the
  projection error, which is the (squared) distance between projected
  points (the points in 3D projected under the normalized camera
  projection matrix $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$) and the
  measured points in normalized coordinates (hint: the error tolerance
  is simpler to calculate in image coordinates using $\boldsymbol{P} =
  \boldsymbol{K} [\boldsymbol{R} | \boldsymbol{t}]$ than in normalized coordinates using
  $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$).

  
  Hint: this problem has codimension 2. 
 

#### Report your values for:
 * the probability $p$ that as least one of the random samples does not contain any outliers
 * the probability $\alpha$ that a given point is an inlier
 * the resulting number of inliers
 * the number of attempts to find the consensus set

In [1]:
import numpy as np
import time

def Homogenize(x):
    # converts points from inhomogeneous to homogeneous coordinates
    return np.vstack((x,np.ones((1,x.shape[1]))))

def Dehomogenize(x):
    # converts points from homogeneous to inhomogeneous coordinates
    return x[:-1]/x[-1]

    
# load data
x0=np.loadtxt('hw3_points2D.txt').T
X0=np.loadtxt('hw3_points3D.txt').T
print('x is', x0.shape)
print('X is', X0.shape)

K = np.array([[1545.0966799187809, 0, 639.5], 
              [0, 1545.0966799187809, 359.5], 
              [0, 0, 1]])

print('K =')
print(K)
    
def ComputeCost(P, x, X, K):
    # Inputs:
    #    P - camera projection matrix 3* 4
    #    x - 2D groundtruth image points
    #    X - 3D groundtruth scene points
    #    K - camera calibration matrix
    #
    # Output:
    #    cost - total projection error in normalized coord
#     n = x.shape[1]
#     covarx = np.eye(2*n) # covariance propagation
    
    num_pts = X.shape[1]
    J_Inv_K = np.linalg.inv(K)[:2, :2]
    cov_norm = J_Inv_K @ np.eye(2) @ J_Inv_K.T # from image to norm coord, cov propagate, 2*2
    
    """your code here"""
#     X_homog = Homogenize(X)
#     x_est_homog = P.dot(X_homog) 
#     x_est_inhomog = Dehomogenize(x_est_homog) # normalized coord est inhomog
    
#     x = np.linalg.inv(K) @ Homogenize(x) # normalized coord measured homog
    
#     err_vect = (Dehomogenize(x) - x_est_inhomog).reshape((-1, 1), order = 'F')
#     cost = err_vect.T @ np.linalg.inv(covarx) @ err_vect
    x_t = Dehomogenize(np.linalg.inv(K) @ Homogenize(x)) # convert image coord pts to normal coord pts, inhomog

    X_p = P @ Homogenize(X) # X estimated from world to camera frame
    x_p = Dehomogenize(X_p) # x estimated in normal frame inhomog, 2D
    
    err_x = x_t - x_p # 2 * num_pts
    cost = 0
    for i in range(num_pts):
        err_xi = err_x[:, i] [:, None]
        cost += err_xi.T @ np.linalg.inv(cov_norm) @ err_xi
    return cost


x is (2, 60)
X is (3, 60)
K =
[[1.54509668e+03 0.00000000e+00 6.39500000e+02]
 [0.00000000e+00 1.54509668e+03 3.59500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [7]:
from scipy.stats import chi2
import random
def ComputePtsErrorVect(P, x, X, K):
    # Inputs:
    #    P - camera projection matrix
    #    x - 2D groundtruth image points
    #    X - 3D groundtruth scene points
    #    K - camera calibration matrix
    #
    # Output:
    #    error_vect image coord

    """your code here"""
    X_homog = Homogenize(X)
    x_est_homog = K @ P.dot(X_homog) # image coord
    x_est_inhomog = Dehomogenize(x_est_homog) # image coord est inhomog
    
   
    err = x - x_est_inhomog
    err_vect = np.sum((err ** 2), axis = 0)
    return err_vect


def ConsensusCost(err_vect, count, tol):
    cost = 0 
    inlier_idxs =  []
    for i in range(count):
        
        if err_vect[i] <= tol:
            cost = cost + err_vect[i]
            inlier_idxs.append(i)  
        else:
            cost = cost + tol
    return cost, inlier_idxs


class FinsterWalder:
    def __init__(self, selected_x, selected_X, K):
        """
        selected_x: inhomog 2d, 2 * 3
        selected_X: inhomog 3d, 3 * 3
        """
        self.x = np.linalg.inv(K) @ Homogenize(selected_x) #selected x in normalized coord homog
        self.X = selected_X

        
    def calculateLength(self):
        P1 = self.X[:, 0]
        P2 = self.X[:, 1]
        P3 = self.X[:, 2]
        
        a = np.linalg.norm(P2 - P3)
        b = np.linalg.norm(P1 - P3)
        c = np.linalg.norm(P1 - P2)
        return a, b, c
    
    
    def calculateUnitVect(self):
        # calculate unit vector in camera coordinate
        j1 = (np.sign(self.x[2, 0]) * self.x[:,0] / np.linalg.norm(self.x[:,0]) )[:, None]
        j2 = (np.sign(self.x[2, 1]) * self.x[:,1] / np.linalg.norm(self.x[:,1]) )[:, None]
        j3 = (np.sign(self.x[2, 2]) * self.x[:,2] / np.linalg.norm(self.x[:,2]) )[:, None]

        return j1, j2, j3
    
    
    @staticmethod
    def calculateCos(j1, j2, j3):
        cos_alpha = j2.T.dot(j3)
        cos_beta = j1.T.dot(j3)
        cos_gamma = j1.T.dot(j2)
        return cos_alpha, cos_beta, cos_gamma
    
    
    def calculateUV(self):
        a, b, c = self.calculateLength()
        j1, j2, j3 = self.calculateUnitVect()
        cos_alpha, cos_beta, cos_gamma = FinsterWalder.calculateCos(j1, j2, j3)
        
        sin_alpha_square = 1 - cos_alpha**2
        sin_beta_square = 1 - cos_beta**2
        sin_gamma_square = 1 - cos_gamma**2
        
        G = c**2 * (c**2 * sin_beta_square - b**2 * sin_gamma_square)
        H = b**2 * (b**2 - a**2) * sin_gamma_square + c**2 * (c**2 + 2 * a**2) * sin_beta_square + \
            + 2 * b**2 * c**2 * (-1 + cos_alpha * cos_beta * cos_gamma )
        I = b**2 * (b**2 - c**2) * sin_alpha_square + a**2 * (a**2 + 2 * c**2) * sin_beta_square + \
            + 2 * a**2 * b**2 * (-1 + cos_alpha * cos_beta * cos_gamma )
        J = a**2 * (a**2 * sin_beta_square - b**2 * sin_alpha_square)
        
        # solving lambdas
        lambdas = np.roots(np.array([G,H,I,J]).reshape(-1))
        real_mask = np.isreal(lambdas)
        lambdas = np.real(lambdas[real_mask])
        
        uvs1_lst = []
#         print(lambdas)
        for lam in lambdas:
            # for each lambda calculate corresponding coeff
            A = 1 + lam
            B = - cos_alpha
            C = ( b**2 - a**2 - lam * c**2 ) / (b**2)
            D = -lam * cos_gamma
            E = ( a**2 + lam * c**2 ) / (b**2) * cos_beta
            F = ( -a**2  + lam * (b**2 - c**2) ) / (b**2)
            
#             p = np.sqrt(B**2 - A * C)
#             q = np.sign(B * E - C * D) * np.sqrt(E**2 - C * F)
#             if (not np.isreal(p)) or (not np.isreal(q)):
#                 # if either p or q is not real, continue for next lam
#                 continue
            if ((B**2 - A * C) < 0) or ((E**2 - C * F) < 0):
                continue
            
            p = np.sqrt(B**2 - A * C)
            q = np.sign(B * E - C * D) * np.sqrt(E**2 - C * F)
        
        
            M = [(-B + p)/C, (-B - p)/C]
            N = [-(E - q)/C, -(E + q)/C]

            # find u and v
            for m, n in zip(M, N):
                A = b**2 - m**2 * c**2
                B = c**2 * (cos_beta - n) * m - b**2 * cos_gamma
                C = -c**2 * n**2 + 2 * c**2 * n * cos_beta + b**2 - c**2
                
                if B**2 - A*C >= 0:
                    u_large = -np.sign(B) * (np.abs(B) + np.sqrt(B**2 - A*C)) / A
                    v_large = u_large * m + n
                    s1_square = a**2 / (u_large**2 + v_large**2 - 2 * u_large * v_large * cos_alpha)
                    if s1_square >= 0:
                        s1 = np.sqrt(s1_square)
                        uvs1_lst.append((u_large, v_large, s1))
   
                        
                    u_small = C / (A * u_large)
                    v_small = u_small * m + n
                   
                    s1_square = a**2 / (u_small**2 + v_small**2 - 2 * u_small * v_small * cos_alpha)
                    if s1_square >= 0:
                        s1 = np.sqrt(s1_square)
                        uvs1_lst.append((u_small, v_small, s1))
          
        return uvs1_lst, j1, j2, j3
    
    
    def calculateCameraCoordPts(self):
        #calculate pts P1 P2 P3 in camera coordinate frame, given uvs1, and j1, j2, j3
        pts_lst = []
        uvs1_lst, j1, j2, j3 = self.calculateUV()
        for u, v, s1 in uvs1_lst:
            s2 = u * s1
            s3 = v * s1
            
            p1 = s1 * j1
            p2 = s2 * j2
            p3 = s3 * j3
            pts_lst.append(np.hstack((p1, p2, p3)))
        return pts_lst
    
def LeastSquareEstimation(world_pts, camera_pts):
    """
    Get R and t, so we can transform world coord to 
    camera coord: camera_pts = R * world_pts + t
    
    world_pts : inhomog 3*n
    camera_pts: inhomog 3*n
    """
    n = world_pts.shape[1]
    mean_world_pts = np.mean(world_pts, axis = 1) [:, None]
    
    mean_camera_pts = np.mean(camera_pts, axis = 1) [:, None]
   
    
    new_world_pts = world_pts - mean_world_pts
    new_camera_pts = camera_pts - mean_camera_pts
    
    cov_camera_world = new_camera_pts.dot(new_world_pts.T) / n
    # col camx camy camz, and row worldx worldy worldz
 
    U, D, Vt = np.linalg.svd(cov_camera_world, full_matrices=True)
    
    S = np.eye(3)
    if np.linalg.det(U) * np.linalg.det(Vt.T) < 0:
        S[2,2] = -1
        
    R = U.dot(S).dot(Vt)
    
    t = mean_camera_pts - R.dot(mean_world_pts)

    return np.hstack((R,t))
    
    
def MSAC(x, X, K, thresh, tol, p):
    # Inputs:
    #    x - 2D inhomogeneous image points
    #    X - 3D inhomogeneous scene points
    #    K - camera calibration matrix
    #    thresh - cost threshold
    #    tol - reprojection error tolerance 
    #    p - probability that as least one of the random samples does not contain any outliers   
    #
    # Output:
    #    consensus_min_cost - final cost from MSAC
    #    consensus_min_cost_model - camera projection matrix P
    #    inliers - list of indices of the inliers corresponding to input data
    #    trials - number of attempts taken to find consensus set
    
    """your code here"""
    trials = 0
    max_trials = np.inf
    consensus_min_cost = np.inf
    consensus_min_cost_model = np.zeros((3,4))
    
    
    total_num_points = x.shape[1]
    s = 3
    trials = 0
    while (trials < max_trials and consensus_min_cost > thresh) :

       
        # select random samples, in this case 3
        selected_sample_idxs = random.sample(range(60), s)
        selected_samples_x = x[:, selected_sample_idxs]
        selected_samples_X = X[:, selected_sample_idxs]
        
        
        # following return a lst of tuple(p1, p2, p3) in camera frame
        finster_walder = FinsterWalder(selected_samples_x, selected_samples_X, K)
        camera_coord_lst_pts = finster_walder.calculateCameraCoordPts()
       
        
        for camera_pts in camera_coord_lst_pts[0:-1]:
            
            model = LeastSquareEstimation(selected_samples_X, camera_pts)
            
            # calculate error for each data point, under each model
            pts_err_vect = ComputePtsErrorVect(model, x, X, K)
            

            # calculate consensus cost
            consensus_cost, inlier_idxs = ConsensusCost(pts_err_vect, total_num_points, tol)
            

            if consensus_cost < consensus_min_cost:
                consensus_min_cost = consensus_cost
                consensus_min_cost_model = model
                
                num_inliers = len(inlier_idxs)
                inliers = inlier_idxs
                w = num_inliers / total_num_points
                max_trials = np.log(1-p)/np.log(1-w**s)
            
        trials = trials + 1
            
    
    # calculate the error for consensus_min_cost_model
    # calculate set of inliers (error <= tol)
    return consensus_min_cost, consensus_min_cost_model, inliers, trials





# MSAC parameters 
thresh = 100
p = 0.99
alpha = 0.95
freedom = 2
tol = chi2.ppf(alpha, freedom) * 1 # inverse chi squared cumulative distribution function * sigma^2 = 1

tic=time.time()
cost_MSAC, P_MSAC, inliers, trials = MSAC(x0, X0, K, thresh, tol, p)
# choose just the inliers
x = x0[:,inliers]
X = X0[:,inliers]

toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
# print('%d iterations'%trials)
# print('inlier count: ',len(inliers))
print('MSAC Cost=%.9f'%cost_MSAC)
print('P = ')
print(P_MSAC)
print('inliers: ',inliers)
print('number of inliers', len(inliers))


took 0.020775 secs
MSAC Cost=170.255151567
P = 
[[  0.28267995  -0.68750784   0.66889836   5.97971745]
 [  0.66118131  -0.36555278  -0.65514154   7.51014406]
 [  0.6949326    0.62745847   0.35123289 175.94286902]]
inliers:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
number of inliers 46


#### Final values for parameters
* $p$ =
* $\alpha$ =
* tolerance = 
* num_inliers =
* num_attempts = 

## Problem 2 (Programming): Estimation of the Camera Pose - Linear Estimate (30 points)
  Estimate the normalized camera projection matrix
  $\hat{\boldsymbol{P}}_\text{linear} = [\boldsymbol{R}_\text{linear} |
  \boldsymbol{t}_\text{linear}]$ from the resulting set of inlier
  correspondences using the linear estimation method (based on the
  EPnP method) described in lecture. Report the resulting 
  $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$.

In [8]:
def EPnP(x, X, K):
    # Inputs:
    #    x - 2D inlier points inhomog
    #    X - 3D inlier points inhomog world
    # Output:
    #    P - normalized camera projection matrix
    
    """your code here"""
    # world frame, calculate control points, which is for deriving alpha1_2_3_4
    n_X = X.shape[1]
    n_x = x.shape[1]
    avg_X = np.mean(X, axis = 1) [:, None]
    cov_X = (X-avg_X).dot((X-avg_X).T) / n_X
    
    U_X, D_X, Vt_X = np.linalg.svd(cov_X, full_matrices=True)
    V_X = Vt_X.T
    
    sigma_square_X = np.trace(cov_X)
    s_X = np.sqrt(sigma_square_X/3)
    
    c1_X = avg_X
    c2_X = s_X * V_X[:, 0] [:, None] + avg_X
    c3_X = s_X * V_X[:, 1] [:, None] + avg_X
    c4_X = s_X * V_X[:, 2] [:, None] + avg_X
    
    A_X = np.hstack((c2_X - c1_X, c3_X - c1_X, c4_X - c1_X))
    new_X = X - c1_X # 3 * N 
    
    # alpha is working in both world and camera frame
    alpha_2_3_4 = np.linalg.inv(A_X) @ new_X
    alpha_1 = (1 - np.sum(alpha_2_3_4, axis = 0))[None, :]
    alpha = np.vstack((alpha_1, alpha_2_3_4))
    
    # camera frame, calculate control points
    M = np.zeros((2 * n_x, 12))
    x_norm_coord = Dehomogenize(np.linalg.inv(K) @ Homogenize(x)) # inhomog
    for i in range(n_x):
        a_i1 = alpha[0, i]
        a_i2 = alpha[1, i]
        a_i3 = alpha[2, i]
        a_i4 = alpha[3, i]
        
        xi = x_norm_coord[0, i]
        yi = x_norm_coord[1, i]
        
        mi = np.array([[a_i1, 0, -a_i1 * xi, a_i2, 0, -a_i2 * xi, a_i3, 0, -a_i3 * xi, a_i4, 0, -a_i4 * xi], 
                       [0, a_i1, -a_i1 * yi, 0, a_i2, -a_i2 * yi, 0, a_i3, -a_i3 * yi, 0, a_i4, -a_i4 * yi]])
        
        M[2*i : 2*i+2, :] = mi
    
    U_x, D_x, Vt_x = np.linalg.svd(M, full_matrices=True) 
    
    # control pts in camera frame
    c_x = Vt_x[-1, :] #get last column of V, or last row of Vt
    c1_x = c_x[0:3][:, None]
    c2_x = c_x[3:6][:, None]
    c3_x = c_x[6:9][:, None]
    c4_x = c_x[9:12][:, None]
    
    X_cam = np.hstack((c1_x, c2_x, c3_x, c4_x)) @ alpha # inhomog 3d pts in camera
    
    # calculate beta scale factor
    avg_X_cam = np.mean(X_cam, axis = 1) [:, None]
    cov_X_cam = (X_cam-avg_X_cam).dot((X_cam-avg_X_cam).T) / n_X
    sigma_square_X_cam = np.trace(cov_X_cam)
    
    
    mask = X_cam[-1, :] >= 0
    mask = mask * 2 - 1
    beta = (mask * np.sqrt(sigma_square_X / sigma_square_X_cam)) [None, :]
    X_cam = X_cam * beta # scale X_cam
    P = LeastSquareEstimation(X, X_cam)
    
#     R = np.eye(3) 
#     t = np.array([[1,0,0]]).T 
#     P = np.concatenate((R, t), axis=1)
    return P

tic=time.time()
P_linear = EPnP(x, X, K) # already be inlier points
toc=time.time()
time_total=toc-tic

cost = ComputeCost(P_linear, x, X, K)
# display the results
print('took %f secs'%time_total)
print('R_linear = ')
print(P_linear[:,0:3])
print('t_linear = ')
print(P_linear[:,-1])
print('cost', cost[0, 0])

took 0.006854 secs
R_linear = 
[[ 0.27861186 -0.69059137  0.66742714]
 [ 0.66108989 -0.36619765 -0.65487361]
 [ 0.69666031  0.62368489  0.35451596]]
t_linear = 
[  5.59648271   7.5120795  175.91580452]
cost 61.25493690106526


## Problem 3 (Programming): Estimation of the Camera Pose - Nonlinear Estimate (30 points) 
  Use $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$ as an
  initial estimate to an iterative estimation method, specifically the
  Levenberg-Marquardt algorithm, to determine the Maximum Likelihood
  estimate of the camera pose that minimizes the projection error
  under the normalized camera projection matrix $\hat{\boldsymbol{P}} =
  [\boldsymbol{R} | \boldsymbol{t}]$.  You must parameterize the camera rotation
  using the angle-axis representation $\boldsymbol{\omega}$ (where
  $[\boldsymbol{\omega}]_\times = \ln \boldsymbol{R}$) of a 3D rotation, which is
  a 3-vector.
  
  
  Report the initial cost (i.e. cost at iteration 0) and the cost at the end
  of each successive iteration. Show the numerical values for the final 
  estimate of the camera rotation $\boldsymbol{\omega}_\text{LM}$ and 
  $\boldsymbol{R}_\text{LM}$, and the camera translation $\boldsymbol{t}_\text{LM}$.

In [9]:
from scipy.linalg import block_diag

# Note that np.sinc is different than defined in class
def Sinc(x):
    # Returns a scalar valued sinc value
    """your code here"""
    if x == 0:
        y = 1
    else:
        y = np.sin(x) / x
    return y

def d_Sinc(x):
    if x == 0:
        return 0
    else:
        return np.cos(x)/x-np.sin(x)/(x**2)

    
def skew(w):
    # Returns the skew-symmetrix represenation of a vector
    """your code here"""
    w = w.ravel()
    w_skew=np.array([[0, -w[2], w[1]],
                     [w[2], 0, -w[0]], 
                     [-w[1], w[0], 0]])
    return w_skew


def Parameterize(R):
    # Parameterizes rotation matrix into its axis-angle representation
    """your code here"""
    
    # solve for v, which is last column of V, or last row of Vt
    U, D, Vt = np.linalg.svd(R-np.eye(3), full_matrices=True)
    v = Vt[-1, :] [:, None]
    # get v tilt
    v_tilt = np.array([[R[2,1] - R[1,2]],
                       [R[0,2] - R[2,0]],
                       [R[1,0] - R[0,1]]])
    # get sin theta
    sin_theta = v.T.dot(v_tilt)/2
    # get cos theta
    cos_theta = (np.trace(R) - 1)/2
    
    # get theta
    theta = np.arctan2(sin_theta, cos_theta) # using arctant2
    # get w
    w = theta * v / np.linalg.norm(v) # 3*1
    if theta > np.pi:
        w = w * (1 - 2*np.pi / theta * np.ceil((theta - np.pi) / (2*np.pi)))
    return w, theta


def Deparameterize(w):
    # Deparameterizes to get rotation matrix
    """your code here"""
    theta = np.linalg.norm(w)
    w_skew = skew(w) # 3*3 matrix
    R = np.cos(theta)*np.eye(3)+Sinc(theta)*w_skew+(1-np.cos(theta))/(theta**2)*w.dot(w.T)
    return R


def Jacobian(R, w, t, X):
    # compute the jacobian matrix
    # Inputs:
    #    R - 3x3 rotation matrix
    #    w - 3x1 axis-angle parameterization of R
    #    t - 3x1 translation vector
    #    X - 3D inlier points inhomog
    #
    # Output:
    #    J - Jacobian matrix of size 2*nx6
    
    """your code here"""
    num_pts = X.shape[1]
    X_rotated = R.dot(X)# in camera frame, but without translation, inhomog
    x_norm = Dehomogenize(X_rotated + t) # in normalized frame, inhomog
    
    J = np.zeros((2*num_pts,6))
    for i in range(num_pts):
        # (2D point inhomog normalized frame)' /(omega)'
        ## (2D point inhomog normalized frame)' /(X_rotated_inhomog)'
        Xi = X_rotated[:, i]
        xi = x_norm[:, i]

        Xi_Z = Xi[-1]
        xi_x = xi[0]
        xi_y = xi[1]
        
        wi_1 = Xi_Z + t[-1, 0]
        
        dx_X_i = np.array([[1/wi_1, 0, -xi_x/wi_1],
                           [0, 1/wi_1, -xi_y/wi_1]])
        ## (X_rotated_inhomog)'/(omega)'
        theta = np.linalg.norm(w)
        v = X[:, i] [:, None]
        if theta<0.00001:
            dX_w_i = skew(-v)
        else:
            s =  (1 - np.cos(theta))/(theta**2)
            ds = ( theta * np.sin(theta) - 2 * (1 - np.cos(theta)) ) / (theta**3)
            
            dX_w_i = Sinc(theta) * skew(-v)\
            + d_Sinc(theta) * np.cross(w.ravel(), v.ravel())[:, None]  @ (1/theta * w.T)\
            + ds * np.cross(w.ravel(), (np.cross(w.ravel(), v.ravel()))) [:, None] @ (1/theta * w.T) \
            + s * (skew(w).dot(skew(-v)) + skew(-np.cross(w.ravel(), v.ravel())[:, None]))

#             s = (1-np.cos(theta))/(theta**2)
#             ds = (theta*np.sin(theta)-2*(1-np.cos(theta)))/(theta**3)
        
#             dX_w_i = Sinc(theta)*skew(-v)+\
#             d_Sinc(theta)*np.cross(w.T,v.T).reshape(3,1).dot(1.0/theta*w.T)\
#             +ds*np.cross(w.T,np.cross(w.T,v.T)).reshape(3,1).dot(1.0/theta*w.T)\
#             +s*np.eye(3).dot(skew(w).dot(skew(-v))+\
#             skew(-np.cross(w.T,v.T).reshape(3,1)))
            
            
            
        dx_w_i = dx_X_i.dot(dX_w_i)
        
        Ai = np.hstack((dx_w_i, dx_X_i))
            
        J[2*i : 2*i+2, :] = Ai
    return J


In [10]:
def LM(P, x, X, K, max_iters, lam):
    # Inputs:
    #    P - initial estimate of camera pose
    #    x - 2D inliers
    #    X - 3D inliers
    #    K - camera calibration matrix 
    #    max_iters - maximum number of iterations
    #    lam - lambda parameter
    #
    # Output:
    #    P - Final camera pose obtained after convergence

    """your code here"""
    num_pts = X.shape[1]
 
    x_t = Dehomogenize(np.linalg.inv(K) @ Homogenize(x)) # convert image coord pts to normal coord pts, inhomog
    J_Inv_K = np.linalg.inv(K)[:2, :2]
    cov_norm = J_Inv_K @ np.eye(2) @ J_Inv_K.T # from image to norm coord, cov propagate, 2*2
    
    X_p = P @ Homogenize(X) # X estimated from world to camera frame
    x_p = Dehomogenize(X_p) # x estimated in normal frame in homog, 2
    
    
    # step 1
    err_x = x_t - x_p # 2 * num_pts
    for j in range(max_iters): # changed i to j
        
        # step 2
        R = P[:3, :3]
        w, _ = Parameterize(R)
        t = P[:, -1] [:, None]
        J = Jacobian(R, w, t, X)
        #print(np.linalg.norm(J))
       
        
        # step 3
        U = 0 # 6 * 6
        E = 0 # 6 * 1
        for i in range(num_pts): 
            err_xi = err_x[:, i] [:, None]
            Ai = J[2*i : 2*i+2, :]
            U += Ai.T @ np.linalg.inv(cov_norm) @ Ai 
            E += Ai.T @ np.linalg.inv(cov_norm) @ err_xi
            
            
        iter_num = 0
        while True:
            if iter_num>20:
                break
            # step 4
            U_lam = U + lam * np.eye(U.shape[1]) # 6 * 6
            delta = np.linalg.inv(U_lam) @ E # 6 * 1
            
            # step 5, canidate
            w_c = w + delta[:3, 0] [:, None]
            R_c = Deparameterize(w_c)
            
            t_c = t + delta[3:, 0] [:, None]
            
            
            # step 6
            P_c = np.hstack((R_c, t_c))
            
            X_p_c = P_c @ Homogenize(X) # X estimated from world to camera frame
            x_p_c = Dehomogenize(X_p_c) # x estimated in normal frame in homog, 2
            
            err_x_c = x_t - x_p_c # 2 * num_pts
            
            #step 7:
            cost_now = 0
            cost_pre = 0
            for i in range(num_pts):
                err_xi_c = err_x_c[:, i] [:, None]
                err_xi = err_x[:, i] [:, None]
                
                cost_now += err_xi_c.T @ np.linalg.inv(cov_norm) @ err_xi_c
                cost_pre += err_xi.T @ np.linalg.inv(cov_norm) @ err_xi
            
            if cost_now < cost_pre: #or abs(cost_now - cost_pre) < 0.000001:
#                 p_est = p_est_c
                err_x = err_x_c
                P = P_c
                lam = 0.1 * lam
                # jump to step 2
                break
            else:
                # jump to step 4, and this is not an iteration
                lam = 10 * lam
                iter_num = iter_num + 1

        cost = ComputeCost(P, x, X, K)
        print('iter %03d Cost %.9f'%(j+1, cost)) # changed i to j
    
    return P
      

# LM hyperparameters
lam = .001
max_iters = 10

tic = time.time()
P_LM = LM(P_linear, x, X, K, max_iters, lam)
w_LM,_ = Parameterize(P_LM[:,0:3])
toc = time.time()
time_total = toc-tic

# display the results
print('took %f secs'%time_total)
print('w_LM = ')
print(w_LM)
print('R_LM = ')
print(P_LM[:,0:3])
print('t_LM = ')
print(P_LM[:,-1])
      
      

iter 046 Cost 61.132239007
iter 046 Cost 61.132228967
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
iter 046 Cost 61.132228966
took 0.226141 secs
w_LM = 
[[ 1.33783933]
 [-0.03144242]
 [ 1.41392914]]
R_LM = 
[[ 0.27845636 -0.69072467  0.6673541 ]
 [ 0.66037579 -0.36684201 -0.65523336]
 [ 0.69739936  0.62315839  0.35398836]]
t_LM = 
[  5.58040783   7.43451112 175.91832973]
